# Implementação - Topic Modelling com LDA

Existem muitas implementações boas de Topic Modelling usando LDA, tanto em Python quanto em R (preferido de David Blei). Assim, o desafio é como extrair boa qualidade de tópicos claros, segregados e significativos.Isto depende muito da qualidade de pre-processamento de texto e da estratégia de encontrar o número ótimo de tópicos. 

Aqui vamos utilizar, mais uma vez, o pacote gensim.

## Importante: Execute esse tutorial no jupyterlab para evitar problemas

In [ ]:
!pip install pprint
!pip install pyLDAvis
# python -m spacy download en - executar pelo terminal

In [ ]:
import nltk; nltk.download('stopwords')
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
# Stop-words: para o conjunto a ser analisado, precisamos adicionar mais algumas stop-words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

Vamos utilizar o 20-Newsgroups dataset para esse exercício. O conjunto é composto por aproximadamente 11k amostras distribuídas em 20 diferentes tópicos

In [ ]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

Pelos exemplos mostrados acima, podemos observar que existem muitos emails, caracter de nova linha (\n) e há também espaços extras. Vamos tratar isso usando regex

In [ ]:
# Converte para lista
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove caracter de nova linha
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove aspas simples
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

O texto ainda está bagunçado e, portanto, não pronto para executar LDA. Precisamos tokenizar e deixá-lo mais limpo. Para isso, vamos usar gensim simple_preprocess.

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True remove pontuação

data_words = list(sent_to_words(data))

print(data_words[:1])

Agora, podemos criar bigramas (duas palavras que frequentemente ocorrem no documento) e trigramas (3 palavras que ocorrem frequentemente)

In [ ]:
# cria modelos bigrama e trigrama
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # quanto maior o threshold menos frases
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Maneira mais rápida de obter uma sentença como um trigrama / bigrama
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# exemplo trigrama
print(trigram_mod[bigram_mod[data_words[0]]])

Agora precisamos remover stop-words, criar bigramas sobre o texto e lematizar

In [ ]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# cria Bigramas
data_words_bigrams = make_bigrams(data_words_nostops)

# inicia o spacy, mantenddo apenas o tagger (eficiência)
nlp = spacy.load('en', disable=['parser', 'ner'])

# faz lemmatização mantendo apenas substantivo, adjetivo, verbo, advérbio
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

Agora precisamos criar o dicionário e o corpus para executar o LDA

In [ ]:
# cria o dicionário
id2word = corpora.Dictionary(data_lemmatized)

# Cria o corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

print(corpus[:1])

Gensim cria um id exclusivo para cada palavra no documento. O corpus produzido mostrado acima é um mapeamento de (word_id, word_frequency).

Por exemplo, (0, 1) acima implica que a palavra cujo id é 0 ocorre uma vez no primeiro documento. Da mesma forma, a palavra id 1 ocorre duas vezes e assim por diante.

Isso é usado como entrada pelo modelo LDA.

Se você quiser ver a palavra que um determinado id corresponde, passe o id como uma chave para o dicionário.

In [ ]:
id2word[0]

Temos tudo o que é necessário para treinar o modelo LDA. Além do corpus e do dicionário, você também precisa fornecer o número de tópicos.

Além disso, alfa e eta são hiperparâmetros que afetam a dispersão dos tópicos. De acordo com os documentos do Gensim, o padrão é 1.0 / num_topics anterior.

chunksize é o número de documentos a serem usados em cada bloco de treinamento. update_every determina com que frequência os parâmetros do modelo devem ser atualizados e passes é o número total de passes de treinamento.

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

O modelo de LDA acima é construído com 20 tópicos diferentes, em que cada tópico é uma combinação de palavras-chave e cada palavra-chave contribui com um certo peso para o tópico.

Você pode ver as palavras-chave de cada tópico e a ponderação (importância) de cada palavra-chave usando lda_model.print_topics () como mostrado a seguir.

In [ ]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

Como interpretar isso?

O tópico 0 é representado como 0.140*"window" + 0.111*"file" + 0.044*"server" + 0.042*"mode" + ' '0.041*"available" + 0.036*"peter" + 0.028*"faq" + 0.027*"information" + ''0.021*"source" + 0.020*"application".

Isso significa que as 10 principais palavras-chave que contribuem para esse tópico são: "window", "file", "server" ... e assim por diante, e o peso de "winddow" no tópico 0 é 0,140.

Os pesos refletem a importância de uma palavra-chave para esse tópico.

Olhando para essas palavras-chave, você pode adivinhar o que este tópico poderia ser? Você pode resumir que são "computador".

In [ ]:
from IPython.display import Image
Image("topics.png")

Perplexidade de modelo e coerência de tópico fornecem uma medida conveniente para julgar quão bom é um determinado modelo de tópico.

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # uma medida de quão bom o modelo é. quanto maiss baaixa, melhor

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

Agora que o modelo de LDA foi criado, a próxima etapa é examinar os tópicos produzidos e as palavras-chave associadas. Não há ferramenta melhor do que o gráfico interativo do pacote pyLDAvis e foi projetado para funcionar bem com os Jupyter notebooks.

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

Então, como inferir a saída do pyLDAvis?

Cada bolha no gráfico do lado esquerdo representa um tópico. Quanto maior a bolha, mais prevalente é esse tópico.

Um bom modelo de tópico terá bolhas grandes e não sobrepostas espalhadas pelo gráfico, em vez de estarem agrupadas em um quadrante.

Um modelo com muitos tópicos geralmente terá muitas sobreposições, bolhas de tamanho pequeno agrupadas em uma região do gráfico.

Ok, se você mover o cursor sobre uma das bolhas, as palavras e barras do lado direito serão atualizadas. Essas palavras são as palavras-chave salientes que formam o tópico selecionado.

De antemão, sabíamos o número ótimo de tópicos, mas e em situações em que essa informação é desconhecida?

Podemos criar uma forma de determinar o melhor número de tópicos (semelhante ao elbow para cluster).

A abordagem para encontrar o número ideal de tópicos é construir muitos modelos de LDA com diferentes valores de número de tópicos (k) e escolher aquele que fornece o maior valor de coerência.

A escolha de um "k" que marca o fim de um rápido crescimento da coerência dos tópicos geralmente oferece tópicos significativos e interpretáveis. Escolher um valor ainda mais alto pode, às vezes, fornecer sub-tópicos mais granulares.

Se você perceber que as mesmas palavras-chave estão sendo repetidas em vários tópicos, provavelmente é um sinal de que o "k" é muito grande.

O compute_coherence_values () (veja abaixo) treina vários modelos de LDA e fornece os modelos e suas pontuações de coerência correspondentes.

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=30, step=6)

In [ ]:
start

In [ ]:
# Visualizando os resultados
limit=30; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
lda_model[corpus[0]]